In [2]:
import cv2
import numpy as np
from tqdm import trange
from flygym import Simulation, Camera
from flygym.arena import FlatTerrain

In this example, we will simulate a fly chasing another fly based on vision. This is similar to the [visual taxis task in week 4](https://github.com/NeLy-EPFL/cobar-exercises/blob/main/week4/1_vision.ipynb), but the ball will be replaced by a walking fly.

For multi-fly simulation, we might want to control the flies differently and have different observation/action spaces for different flies. Therefore, we will inherit from `Fly` and implement different logic in the `pre_step` method. 

In [3]:
from hybrid_turning_fly import HybridTurningFly

In [4]:
timestep = 2e-4

fly0 = HybridTurningFly(
    name="0",
    timestep=timestep,
    enable_adhesion=True,
    spawn_pos=(0, 0, 0),
)

fly1 = HybridTurningFly(
    name="1",
    timestep=timestep,
    enable_adhesion=True,
    spawn_pos=(6, 0, 0.5),
)

arena = FlatTerrain()

# THIS IS THE CAMERA THAT WILL BE USED FOR FOLLOWING THE CENTER OF MASS OF THE FLY
birdeye_cam = arena.root_element.worldbody.add(
    "camera",
    name="birdeye_cam",
    mode="fixed",
    pos=(15, 0, 40),
    euler=(0, 0, 0),
    fovy=45,
)
#-------------------------------------------------------------------------------

cam = Camera(
    fly=fly0,
    camera_id="Animat/camera_left", # This is not used as argument
    play_speed=0.5,
    window_size=(800, 608),
)


sim = Simulation(
    flies=[fly0, fly1],
    cameras=[cam],
    arena=arena,
    timestep=timestep,
)

In [5]:
sim.reset(seed=0)
for i in fly1.model.find_all("geom"):
    sim.physics.named.model.geom_rgba[f"1/{i.name}"] = (0, 0, 0, 1)

run_time = 1
t = np.arange(0, run_time, timestep)

fly0_actions = (
    np.column_stack(
        [
            np.abs(np.cos(t * np.pi / 2)),
            np.abs(np.sin(t * np.pi / 2)),
        ]
    )
    * 1.2
)
fly1_actions = (
    np.column_stack(
        [
            np.abs(np.cos(t * np.pi / 2)),
            np.abs(np.sin(t * np.pi / 2)),
        ]
    )
    * 1.2
)

second_cam_frames = []
x = None
y = None
alpha = 1e-1

for i in trange(len(t)):
    obs, _, _, _, info = sim.step(
        {
            "0": fly0_actions[i],
            "1": fly1_actions[i],
        }
    )

    obs0, info0 = obs["0"], info["0"] # not used




    #THIS PART IS TO MAKE THE SECOND CAMERA FOLLOW THE CENTER OF MASS OF THE FLY ------------------------------------------------
    render_res = sim.render()[0]

    if render_res is not None:
        second_cam = sim.physics.bind(birdeye_cam)

        x_new = sim._get_center_of_mass()[0]
        y_new = sim._get_center_of_mass()[1]

        if x is None:
            x = x_new
        
        if y is None:
            y = y_new

        x = (1 - alpha) * x + alpha * x_new
        y = (1 - alpha) * y + alpha * y_new

        second_cam.pos[0] = x
        second_cam.pos[1] = y

        second_img = sim.physics.render(
            width=700, height=560, camera_id="birdeye_cam"
        )
        second_img = cv2.putText(
            np.ascontiguousarray(second_img),
            f"{sim.cameras[0].play_speed}x",
            org=(20, 30),
            fontFace=cv2.FONT_HERSHEY_DUPLEX,
            fontScale=0.8,
            color=(0, 0, 0),
            lineType=cv2.LINE_AA,
            thickness=1,
        )
        second_cam_frames.append(second_img)
        #------------------------------------------------------------------------------------------------



100%|██████████| 5000/5000 [01:33<00:00, 53.64it/s]


In [6]:
#birdeye_cam_frames = cam._frames
cam._frames = second_cam_frames
cam.save_video("test.mp4")